In [61]:
from sqlalchemy import create_engine, text
import pymysql
import pandas as pd
import plotly.express as ex

In [93]:
user = "bigdata"
password = "Bigdata123!!"
host = "192.168.56.101"
port = "3306"
db = "SampleDB"
connect_script = f'mysql+pymysql://{user}:{password}@{host}:{port}/{db}'
engine = create_engine(connect_script)

In [63]:
conn = engine.connect()

In [92]:
qry = """
select *
 from 학생평가
"""
pd.read_sql(qry,conn)

,학번,이름,점수,조이름
0,201501001,김수현,90,T2015
1,201501002,송혜교,75,무한도전
2,201501003,하지원,77,학교짱
3,201501004,장근석,85,T2015
4,201501005,이종석,96,BIG
5,201501006,주원,80,베스트
6,201501007,박보영,95,BIG
7,201501008,한효주,90,학교짱
8,201501009,송중기,83,T2015
9,201501010,김우빈,88,베스트


In [5]:
stmt = """
insert into 학생평가(
학번, 
이름, 
점수, 
조이름) 
values(
201501016,
'홍길동',
100,
'빅데이터'
)
"""
conn.execute(text(stmt))

In [9]:
qry = """
select a.*
 from 학생평가 a, (select max(점수) as 점수 from 학생평가) b
where a.점수 = b.점수
"""
pd.read_sql(qry, conn)

,학번,이름,점수,조이름
0,201501016,홍길동,100,빅데이터


In [13]:
df = ex.data.gapminder()

In [15]:
df.dtypes

country       object
continent     object
year           int64
lifeExp      float64
pop            int64
gdpPercap    float64
iso_alpha     object
iso_num        int64
dtype: object

In [17]:
stmt = """
create table world
(country varchar(50),
 continent varchar(30),
 year int,
 pop int
)
"""
conn.execute(text(stmt))

In [18]:
pd.read_sql("select * from world", conn)

,country,continent,year,pop


In [25]:
x = df.loc[[0],["country","continent","year","pop"]].to_dict('r')[0]

/tmp/ipykernel_1855/821947718.py:1: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  x = df.loc[[0],["country","continent","year","pop"]].to_dict('r')[0]


In [26]:
x

{'country': 'Afghanistan', 'continent': 'Asia', 'year': 1952, 'pop': 8425333}

In [27]:
stmt = """
insert into world(
country,
continent,
year,
pop
)
values(
:country,
:continent,
:year,
:pop
)
"""
conn.execute(text(stmt),**x)

In [28]:
pd.read_sql("select * from world", conn)

,country,continent,year,pop
0,Afghanistan,Asia,1952,8425333


In [56]:
# 학번 = input("학번 >>>")
name = input("이름 >>>")
score = int(input("점수 >>>"))

stmt = """
select max(학번) as max
 from 학생평가
"""
st_id = str(int(pd.read_sql(stmt,conn).loc[:,'max'][0])+1)  # 학번이 자동으로 1씩 증가하면서 입력됨

stmt = """
insert into 학생평가(
학번,
이름,
점수
)
values(
:x1,
:x2,
:x3
)
"""
df1 = pd.read_sql("select count(*) as cnt from 학생평가", conn)
df1 = df1.loc[:,'cnt'][0]
dt = {"x1":st_id, "x2":name, "x3":score}
conn.execute(text(stmt),**dt)
df = pd.read_sql("select count(*) as cnt from 학생평가", conn)
df = df.loc[:,'cnt'][0]
if df > df1:
    print(f'{name} 학생 정보의 입력이 잘 되어습니다')

이름 >>> 최영희
점수 >>> 97


최영희 학생 정보의 입력이 잘 되어습니다


In [67]:
df1 = pd.read_sql("select count(*) as cnt from 학생평가", conn)
df1.loc[:,'cnt'][0] == 16

False

In [65]:
name = input("조회하려는 학생 이름을 입력하세요 >>>")
qry = f"select * from 학생평가 where 이름 = '{name}'"
df = pd.read_sql(qry,conn)
if len(df) > 0:
    print(df)
else:
    print("조회하려는 학생이 없습니다")

조회하려는 학생 이름을 입력하세요 >>> 최영희


조회하려는 학생이 없습니다


In [66]:
stmt = """
select max(학번) as max
 from 학생평가
"""
str(int(pd.read_sql(stmt,conn).loc[:,'max'][0])+1)

'201501018'

In [99]:
## 학생평가1 테이블을 생성 "create table 학생평가1 like 학생평가"
## 학생평가1 인서트
## insert into 학생평가1() values() conn.execute(text(stmt))
## df1.to_dict('r')

# 학생평가 테이블을 받아와서 학생평가1 테이블 작성
stmt = """
create table 학생평가1 like 학생평가
"""
conn.execute(text(stmt))

In [103]:
df = pd.read_sql("select * from 학생평가", conn)  # 학생평가 테이블 로드

df1 = df.loc[(df.점수 % 3 == 0) & (df.점수 % 5 == 0),:] # 로드한 학생평가 테이블 중 점수가 3, 5 배수인 행만 추출

In [104]:
df1

,학번,이름,점수,조이름
0,201501001,김수현,90,T2015
1,201501002,송혜교,75,무한도전
7,201501008,한효주,90,학교짱
10,201501011,이연희,75,학교짱


In [106]:
df1.to_dict('r') # r 또는 records, 추출한 데이터를 딕셔너리형으로 변환

/tmp/ipykernel_1855/3723802149.py:1: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  df1.to_dict('r') # r 또는 records


[{'학번': '201501001', '이름': '김수현', '점수': 90, '조이름': 'T2015'},
 {'학번': '201501002', '이름': '송혜교', '점수': 75, '조이름': '무한도전'},
 {'학번': '201501008', '이름': '한효주', '점수': 90, '조이름': '학교짱'},
 {'학번': '201501011', '이름': '이연희', '점수': 75, '조이름': '학교짱'}]

In [107]:
insert_stmt = """
insert into 학생평가1(
학번,
이름,
점수,
조이름)
values(
:학번,
:이름,
:점수,
:조이름
)
"""

for dt in df1.to_dict('r'):    # df1.to_dict('r') 안의 데이터를 받아와 dt에 삽입
    print(dt)
    conn.execute(text(insert_stmt),**dt)

{'학번': '201501001', '이름': '김수현', '점수': 90, '조이름': 'T2015'}
{'학번': '201501002', '이름': '송혜교', '점수': 75, '조이름': '무한도전'}
{'학번': '201501008', '이름': '한효주', '점수': 90, '조이름': '학교짱'}
{'학번': '201501011', '이름': '이연희', '점수': 75, '조이름': '학교짱'}


/tmp/ipykernel_1855/715507070.py:15: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  for dt in df1.to_dict('r'):


In [109]:
stmt = """
select *
 from 학생평가1
"""
pd.read_sql(text(stmt),conn)

,학번,이름,점수,조이름
0,201501001,김수현,90,T2015
1,201501002,송혜교,75,무한도전
2,201501008,한효주,90,학교짱
3,201501011,이연희,75,학교짱


In [114]:
stmt = """
select *
 from 학생평가
where 점수 %% 3 = 0
  and 점수 %% 5 = 0
"""
pd.read_sql(stmt,conn)

sum(pd.read_sql("select * from 학생평가1",conn).점수) == sum(pd.read_sql(stmt,conn).점수)

True